In [126]:
 # Source code: https://meshlogic.github.io/posts/jupyter/curve-fitting/parametric-curve-fitting/

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
import numpy as np
from numpy.polynomial import polynomial as pl
import os
import math
import numpy as np
from numpy import linalg, linspace, zeros
from math import ceil, log

In [127]:
#function to read a file with coordinates and extract the x, y, and z coordinates for each file
def read_xyz_file(filename):
    with open(filename, 'r') as file:
        coordinates = [list(map(float, line.split(","))) for line in file]
        # Splitting the line by whitespace and converting values to float
    x_coordinates, y_coordinates, z_coordinates = zip(*coordinates)

    return x_coordinates, y_coordinates, z_coordinates

# This function finds distance between two points in 3D space
def finddistance(x,y,z,x1,y1,z1):
    return math.sqrt((x-x1)**2 + (y-y1)**2 + (z-z1)**2)


In [128]:
def uniform_param(P):
    u = linspace(0, 1, len(P))
    return u

def chordlength_param(P):
    u = generate_param(P, alpha=1.0)
    return u

def centripetal_param(P):
    u = generate_param(P, alpha=0.5)
    return u

# This function generates parameters for a curve based on a given power
def generate_param(P, alpha):
    n = len(P)
    u = zeros(n)
    u_sum = 0
    for i in range(1,n):
        u_sum += linalg.norm(P[i,:]-P[i-1,:])**alpha
        u[i] = u_sum
    return u/max(u)

# This function performs the golden section search to find minimum of a function within an interval
def find_min_gss(f, a, b, eps=1e-4):

    R = 0.61803399
    n_iter = int(ceil(-2.0780869 * log(eps/abs(b-a))))
    c = b - (b-a)*R
    d = a + (b-a)*R

    for i in range(n_iter):
        if f(c) < f(d):
            b = d
        else:
            a = c
        c = b - (b-a)*R
        d = a + (b-a)*R
    return (b+a)/2

# This function generates iterative parameters for curve fitting and visualizes the error in approximation
def iterative_param(P, u, fxcoeff, fycoeff, fzcoeff, fig_ax):

    global iter_i, plt_color
    u_new = u.copy()
    f_u = zeros(3)

    def calc_s(u):
        f_u[0] = np.polyval(fxcoeff, u)
        f_u[1] = np.polyval(fycoeff, u)
        f_u[2] = np.polyval(fzcoeff, u)
        s_u = linalg.norm(P[i]-f_u)
        return s_u

    for i in range(1, len(u)-1):
        u_new[i] = find_min_gss(calc_s, u[i-1], u[i+1])
        u_samp = linspace(u[i-1], u[i+1], 25)
        x = np.polyval(fxcoeff, u_samp)
        y = np.polyval(fycoeff, u_samp)
        z = np.polyval(fzcoeff, u_samp)
        residual = P[i] - np.array([x,y,z]).T
        s_u_samp = [linalg.norm(residual[j]) for j in range(len(u_samp))]
        fig_ax.plot(u_samp, s_u_samp, color=plt_color[iter_i], alpha=plt_alpha)
        fig_ax.plot(u_new[i], calc_s(u_new[i]), 'o', color=plt_color[iter_i], alpha=plt_alpha)

    return u_new

In [129]:
#splits the coordinates of phosphorous atoms into two sets of coordinates - x, y, and z coords for each helix
def process_coordinates(directory, filename):
    filepath = os.path.join(directory, filename)
    xs, ys, zs = read_xyz_file(filepath)

    print_coords('x', xs)
    print_coords('y', ys)
    print_coords('z', zs)

    middle_elementx = len(xs) // 2

    endindex = middle_elementx
    if len(xs) % 2 == 1 :
        distance1 = finddistance(xs[middle_elementx], ys[middle_elementx], zs[middle_elementx], xs[middle_elementx-1], ys[middle_elementx-1], zs[middle_elementx-1])
        distance2 = finddistance(xs[middle_elementx], ys[middle_elementx], zs[middle_elementx], xs[middle_elementx+1], ys[middle_elementx+1], zs[middle_elementx+1])
        if distance1 < distance2:
            endindex += 1

    x, x1 = split_and_print_coords('x', xs, endindex)
    y, y1 = split_and_print_coords('y', ys, endindex)
    z, z1 = split_and_print_coords('z', zs, endindex)

    return x, y, z, x1, y1, z1

def print_coords(name, coords):
    print(f'Number of {name} coordinates: {len(coords)}')
    print(f'{name} coordinates: {coords}')

def split_and_print_coords(name, coords, index):
    first_half = coords[:index]
    second_half = coords[index:]

    print_coords(f'{name} first half', first_half)
    print_coords(f'{name} second half', second_half)

    return first_half, second_half


In [ ]:
#determining the total distance covered by the connection between the knot points and the middle distance
def determine_midpoint(x, y, z):
    distance_list = []
    total_distance = 0
    for x_coord in x:
        x_temp = float(x_coord)
        y_temp = float(y[x.index(x_coord)])
        z_temp = float(z[x.index(x_coord)])
        if (x.index(x_coord) + 1) != len(x):
            x_temp1 = float(x[x.index(x_coord) + 1])
            y_temp1 = float(y[x.index(x_coord) + 1])
            z_temp1 = float(z[x.index(x_coord) + 1])

            distance = ((x_temp1 - x_temp)**2 + (y_temp1 - y_temp)**2 + (z_temp1 - z_temp)**2)**0.5
            distance_list.append(distance)
    
    for distance in distance_list:
        total_distance += distance

    half_length = total_distance/2
    print(half_length)

    distance_from_closest_point = half_length
    point_1x = 0
    point_1y = 0
    point_1z = 0
    point_2x = 0
    point_2y = 0
    point_2z = 0

    for distance in distance_list:
        distance_from_closest_point -= distance
        if distance_from_closest_point < 0:
            distance_from_closest_point += distance
            point_1x = float(x[distance_list.index(distance)])
            point_1y = float(y[distance_list.index(distance)])
            point_1z = float(z[distance_list.index(distance)])
            point_2x = float(x[distance_list.index(distance) + 1])
            point_2y = float(y[distance_list.index(distance) + 1])
            point_2z = float(z[distance_list.index(distance) + 1])
            break
        else:
            continue

    print("(" + str(point_1x) + "," + str(point_1y) + "," + str(point_1z) + "), (" + str(point_2x) + "," + str(point_2y) + "," + str(point_2z) + ")")

    distance_between_point = ((point_2x - point_1x)**2 + (point_2y - point_1y)**2 + (point_2z - point_1z)**2)**0.5

    r = distance_from_closest_point/distance_between_point

    dx = point_2x-point_1x
    rdx = abs(r * dx)
    mid_x = rdx + point_1x

    dy = point_2y-point_1y
    rdy = abs(r * dy)
    mid_y = rdy + point_1y

    dz = point_2z-point_1z
    rdz = abs(r * dz)
    mid_z = rdz + point_1z
    print("(" + str(mid_x) + "," + str(mid_y) + "," + str(mid_z) + ")")
    return mid_x, mid_y, mid_z


In [ ]:
def final_midpoint(x, y, z, x1, y1, z1)
    final_mid_x = mid_x + mid_x1
    final_mid_x *= 0.5
    final_mid_y = mid_y + mid_y1
    final_mid_y *= 0.5
    final_mid_z = mid_z + mid_z1
    final_mid_z *= 0.5

    print("(" + str(final_mid_x) + "," + str(final_mid_y) + "," + str(final_mid_z) + ")")
    return final_mid_x, final_mid_y, final_mid_z

In [ ]:
def find_radius(x, y, z, x1, y1, z1, final_mid_x, final_mid_y, final_mid_z)
    radius_helix_1 = 100000
    radius_helix_2 = 100000

    for coordinate in x:
        x_temp_rad = float(coordinate)
        y_temp_rad = float(y[x.index(coordinate)])
        z_temp_rad = float(z[x.index(coordinate)])
        distance = ((final_mid_x - x_temp_rad)**2 + (final_mid_y - y_temp_rad)**2 + (final_mid_z - z_temp_rad)**2)**0.5
        if distance < radius_helix_1:
            radius_helix_1 = distance

    for coordinate1 in x1:
        x1_temp_rad = float(coordinate)
        y1_temp_rad = float(y[x.index(coordinate1)])
        z1_temp_rad = float(z[x.index(coordinate1)])
        distance = ((final_mid_x - x1_temp_rad)**2 + (final_mid_y - y1_temp_rad)**2 + (final_mid_z - z1_temp_rad)**2)**0.5
        if distance < radius_helix_2:
            radius_helix_2 = distance


    final_radius = (radius_helix_1 + radius_helix_2)/2
    return final_radius

In [ ]:
def print_and_write_to_file(filename, final_radius):
  print(final_radius)

  try:
    with open("radius.txt", "a") as f:
      f.write(f"{filename}, {final_radius}")
  except IOError:
    print("An error occurred trying to write the file.")


In [ ]:
f = open("coefficients.txt", "w")
directory = r"C:\Users\magne\Downloads\DNA_Coords\DNA_Coords"
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
            x, y, z, x1, y1, z1 = process_coordinates(directory, filename)
            mid_x, mid_y, mid_z = determine_midpoint(x, y, z)
            mid_x1, mid_y1, mid_z1 = determine_midpoint(x1, y1, z1)
            final_mid_x, final_mid_y, final_mid_z = final_midpoint(x, y, z, x1, y1, z1)
            final_radius = find_radius(x, y, z, x1, y1, z1, final_mid_x, final_mid_y, final_mid_z)
            print_and_write_to_file(filename, final_radius)